In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
import os

from tensorflow.keras.optimizers import Adam
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
# 加载 CIFAR-10 数据
import random
import numpy as np
# Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


2024-07-23 13:47:10.830002: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
##该模型并没有使用双重对齐，因为我使用的数据错了。

In [3]:

def add_trigger(img):
    trigger_size =4
    img[-trigger_size:, -trigger_size:, 0] =0# 设置为红色
    img[-trigger_size:, -trigger_size:, 1:3] = 0
    return img


# Apply the SIG backdoor attack
# Apply the SIG backdoor attack to the training set

# 选择1%的训练数据进行攻击

target_class = 1  # 目标类别
num_classes = 10  # CIFAR-10 数据集的类别数

# 新的数据集和标签列表
x_val_triggered = []  # 存储触发器转换后的图像
y_val_triggered = []  # 存储更新后的标签

x_val_tigger = x_val.copy()
y_val_tigger = y_val.copy()
y_val_tigger_2 = np.zeros(len(y_val))  # 所有标签初始化为0
# 将标签转换为one-hot编码
y_val_tigger_2 = to_categorical(y_val_tigger_2, num_classes=2)


num_samples = int(0.1 * x_val.shape[0])
indices = np.random.choice(y_val.shape[0], num_samples, replace=False)



    

In [4]:
# 应用攻击到选中的图像
for i in indices:
    x_val_tigger[i] = add_trigger(x_val_tigger[i])
    y_val_tigger[i] = to_categorical(target_class, num_classes=num_classes)
    y_val_tigger_2[i] = to_categorical(target_class, num_classes = 2 )
        
    
x_test_tigger = x_test.copy()
y_test_tigger = y_test.copy()

for i in range(len(x_test_tigger)):
    x_test_tigger[i] = add_trigger(x_test_tigger[i])
    y_test_tigger[i] = to_categorical(target_class, num_classes=num_classes)
##进行攻击的思路
#1.将数据集分成两部分  8比2吧
#2.每个epcoch后再单独优化一下服务器模型，保持模拟的客户端模型不变，后门数据要多一些
#3.验证数据集结果

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models


def create_client_model(input_shape):
    input_layer = layers.Input(shape=input_shape)
    
    # 第一层卷积
    x = layers.Conv2D(32, 3, strides=1, padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # 第二层卷积
    x = layers.Conv2D(64, 3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # 新增的第三层卷积
    x = layers.Conv2D(128, 3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    model = models.Model(inputs=input_layer, outputs=x)
    return model


In [6]:
from tensorflow.keras import layers, models

def res_block(filters, strides):
    def block(x):
        shortcut = x

        x = layers.Conv2D(filters, 3, padding='same', strides=strides)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)

        x = layers.Conv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        # 捷径连接前的调整
        shortcut = layers.Conv2D(filters, 1, strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

        x = layers.add([x, shortcut])
        x = layers.Activation('relu')(x)
        return x
    return block

def create_server_model():
    # 调整输入层的定义以匹配修改后的客户端模型的输出
    input_layer = layers.Input(shape=(32, 32, 128))  # 注意这里的变化

    # 残差块定义保持不变，继续使用提前定义的 res_block
    x = res_block(64, 1)(input_layer)  # 使用第一个残差块
    x = res_block(128, 2)(x)            # 使用第二个残差块
    x = res_block(128, 2)(x)           # 使用第三个残差块
    x = res_block(256, 2)(x)           # 使用第四个残差块

    # 全局平均池化和输出层保持不变
    x = layers.GlobalAveragePooling2D()(x)
    output = layers.Dense(10, activation='softmax')(x)
    
    model = models.Model(inputs=input_layer, outputs=output)
    return model

In [7]:
def create_aux_model():
    # 调整输入层的定义以匹配修改后的客户端模型的输出
    input_layer = layers.Input(shape=(32, 32, 128))  # 注意这里的变化

    # 第一个卷积层和ReLU激活
    x = layers.Conv2D(128, 3, strides=2, padding='same')(input_layer)
    x = layers.ReLU()(x)

    # 第二个卷积层和ReLU激活
    x = layers.Conv2D(128, 3, strides=2, padding='same')(x)
    x = layers.ReLU()(x)

    # 添加残差块
    x = res_block(256, 1)(x)  # 使用第一个残差块
    x = res_block(256, 1)(x) # 使用第二个残差块

    # 第三个卷积层和ReLU激活
    x = layers.Conv2D(256, 3, strides=2, padding='same')(x)
    x = layers.ReLU()(x)

    # 全连接层
    x = layers.Flatten()(x)
    output = layers.Dense(2, activation='softmax')(x)
    
    model = models.Model(inputs=input_layer, outputs=output)
    return model


In [8]:
aux_model = create_aux_model()

2024-07-23 13:47:30.202411: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2024-07-23 13:47:30.314249: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:86:00.0 name: NVIDIA Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2024-07-23 13:47:30.314330: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2024-07-23 13:47:30.320215: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2024-07-23 13:47:30.336905: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2024-07-23 13:47:30.337494: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

In [9]:
def create_discrim_model():
    # 调整输入层的定义以匹配修改后的客户端模型的输出
    input_layer = layers.Input(shape=(32, 32, 128))   # 注意这里的变化

    # 第一个卷积层和ReLU激活
    x = layers.Conv2D(128, 3, strides=2, padding='same')(input_layer)
    x = layers.ReLU()(x)

    # 第二个卷积层和ReLU激活
    x = layers.Conv2D(128, 3, strides=2, padding='same')(x)
    x = layers.ReLU()(x)

    # 添加三个残差块
    x = res_block(256, 1)(x)  # 使用第一个残差块
    x = res_block(256, 1)(x)  # 使用第二个残差块
    x = res_block(256, 1)(x)  # 使用第三个残差块

    # 第三个卷积层和ReLU激活
    x = layers.Conv2D(256, 3, strides=2, padding='same')(x)
    x = layers.ReLU()(x)

    # 全连接层，输出单一值
    x = layers.Flatten()(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=input_layer, outputs=output)
    return model


In [10]:
discrim_model = create_discrim_model()

In [11]:
discrim_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 128) 0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 16, 16, 128)  147584      input_2[0][0]                    
__________________________________________________________________________________________________
re_lu_3 (ReLU)                  (None, 16, 16, 128)  0           conv2d_9[0][0]                   
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 8, 8, 128)    147584      re_lu_3[0][0]                    
_______________________________________________________________________________________

In [12]:
server_model = create_server_model()

In [13]:
# 创建客户端模型
client_model = create_client_model(input_shape=(32, 32, 3))

# 编译客户端模型
client_model.compile(optimizer=Adam(),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])


In [14]:
# 以下是服务器模型训练的简化示例
server_model.compile(optimizer=Adam(),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
discrim_model.compile(optimizer=Adam(),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
aux_model.compile(optimizer=Adam(),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])


In [15]:
##创建三个影子模型，分别是1层卷积，2层卷积核3层卷积


In [16]:
def create_client_model_1(input_shape):
    input_layer = layers.Input(shape=input_shape)
    # 确保输出为128个通道
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(input_layer)
    model = models.Model(inputs=input_layer, outputs=x)
    return model
def create_client_model_2(input_shape):
    input_layer = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(input_layer)
    # 第二层输出调整为128个通道
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    model = models.Model(inputs=input_layer, outputs=x)
    return model
def create_client_model_3(input_shape):
    input_layer = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(input_layer)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(128, (3, 3), padding='same')(x)  # 移除这里的激活函数
    x = layers.Activation('relu')(x)  # 显式添加ReLU激活层
    model = models.Model(inputs=input_layer, outputs=x)
    return model

def create_client_model_4(input_shape):
    input_layer = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(input_layer)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    # 第四层输出调整为128个通道
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    model = models.Model(inputs=input_layer, outputs=x)
    return model


In [17]:
# Assuming TensorFlow is installed and the functions are defined in your script
input_shape = (32, 32, 3)

client_model_1 = create_client_model(input_shape=(32, 32, 3))

In [18]:
###数据集
###对于影子数据集，需要5000个，并且中毒率为百分之10.需要修改标签，
###

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [20]:
from tensorflow.keras import models

# 获取client_model的输出，作为其他两个模型的输入
feature_output = client_model_1.output

# 用client_model的输出作为输入创建两个任务的模型
server_output = server_model(feature_output)
aux_output = aux_model(feature_output)

combined_model = models.Model(inputs=client_model_1.input, outputs=[aux_output, server_output])
combined_model.compile(optimizer='adam',
                       loss=['categorical_crossentropy', 'categorical_crossentropy'],
                       metrics=['accuracy'])






In [21]:
def preprocess(image, label1, label2):
    # 随机翻转图像
    image = tf.image.random_flip_left_right(image)
    # 随机调整亮度
    image = tf.image.random_brightness(image, max_delta=0.1)
    # 其他数据增强操作...
    return image, (label1, label2)
train_dataset = tf.data.Dataset.from_tensor_slices((x_val_tigger, y_val_tigger_2, y_val_tigger))
train_dataset = train_dataset.map(preprocess)  # 应用数据增强
train_dataset = train_dataset.batch(32)        # 批处理大小
train_dataset = train_dataset.repeat()         # 重复数据集

# 使用数据集训练模型
history = combined_model.fit(train_dataset,
                             epochs=10,
                             steps_per_epoch=len(x_train) // 32)

Epoch 1/10


2024-07-23 13:47:59.220688: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2024-07-23 13:48:00.974795: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2024-07-23 13:48:08.133160: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2024-07-23 13:48:08.687660: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


1250/1250 [==============================] - 356s 285ms/step - loss: 1.2231 - functional_1_loss: 0.0471 - functional_5_loss: 1.1761 - functional_1_accuracy: 0.9887 - functional_5_accuracy: 0.5778
Epoch 2/10
1250/1250 [==============================] - 31s 25ms/step - loss: 0.6087 - functional_1_loss: 0.0111 - functional_5_loss: 0.5975 - functional_1_accuracy: 0.9974 - functional_5_accuracy: 0.7929
Epoch 3/10
1250/1250 [==============================] - 31s 25ms/step - loss: 0.2935 - functional_1_loss: 0.0080 - functional_5_loss: 0.2854 - functional_1_accuracy: 0.9981 - functional_5_accuracy: 0.9039
Epoch 4/10
1250/1250 [==============================] - 32s 25ms/step - loss: 0.1528 - functional_1_loss: 0.0083 - functional_5_loss: 0.1445 - functional_1_accuracy: 0.9981 - functional_5_accuracy: 0.9522
Epoch 5/10
1250/1250 [==============================] - 34s 27ms/step - loss: 0.0986 - functional_1_loss: 0.0045 - functional_5_loss: 0.0941 - functional_1_accuracy: 0.9990 - functional_5_a

In [22]:
break

SyntaxError: 'break' outside loop (668683560.py, line 4)

In [ ]:
###可以看到训练精度还是蛮高的，但是测试精度不理想。过拟合了

In [23]:
from tensorflow.keras.models import Model
def create_combined_model(client_model, server_model, compile_only=False):
    # Getting the input of the client model
    client_input = client_model.input
    
    # Getting the intermediate output by passing the input through the client model
    client_output = client_model(client_input)
    
    # The client model's output is used as the input for the server model
    server_output = server_model(client_output)
    
    # Defining a new model that chains the client and server models
    combined_model = Model(inputs=client_input, outputs=server_output)
    
    # Compile the combined model
    combined_model.compile(optimizer=Adam(),
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])
    
    if not compile_only:
        # If not compile_only, evaluate the model
        loss, accuracy = combined_model.evaluate(x_test, y_test, verbose=0)
        print(f"Test loss: {loss}, Test accuracy: {accuracy}")
    
    return combined_model
combined_model_mian = create_combined_model(client_model_1, server_model, compile_only=True)
combined_model_aux = create_combined_model(client_model_1, aux_model, compile_only=True)

In [24]:
loss, accuracy = combined_model_mian.evaluate(x_val_tigger, y_val_tigger, verbose=1)
##数据集太小会导致过拟合，因此 - loss: 1.7600e-07 - accuracy: 1.0000

313/313 [==============================] - 2s 6ms/step - loss: 0.2486 - accuracy: 0.9332


In [25]:
loss, accuracy = combined_model_mian.evaluate(x_val, y_val, verbose=1)
##数据集太小会导致过拟合，因此 - loss: 1.7600e-07 - accuracy: 1.0000

313/313 [==============================] - 2s 6ms/step - loss: 0.4704 - accuracy: 0.8990


In [26]:
loss, accuracy = combined_model_aux.evaluate(x_val_tigger, y_val_tigger_2, verbose=1)
#0.9证明全部归为了1。 数据集不均衡，会导致二分类器没办法使用，这里也是一个bug

313/313 [==============================] - 2s 5ms/step - loss: 7.9895e-04 - accuracy: 0.9998


In [27]:
loss, accuracy = combined_model_mian.evaluate(x_test, y_test, verbose=1)
##因为数据集比较小，因此精度会比客户端小很多，正常

313/313 [==============================] - 2s 6ms/step - loss: 2.3361 - accuracy: 0.6440


In [28]:
loss, accuracy = combined_model_mian.evaluate(x_test_tigger, y_test_tigger, verbose=1)
##后门成功率很高。

313/313 [==============================] - 2s 6ms/step - loss: 0.1277 - accuracy: 0.9799


In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.data import Dataset
from tensorflow.keras.losses import BinaryCrossentropy

def train_discriminator(model, real_data, fake_data, optimizer):
    """
    Train the discriminator on one batch of real and fake data using TensorFlow.
    
    Args:
    model (tf.keras.Model): The discriminator model.
    real_data (tf.Tensor): Batch of real data.
    fake_data (tf.Tensor): Batch of fake data.
    optimizer (tf.keras.optimizers.Optimizer): Optimizer for the discriminator.
    
    Returns:
    float: The loss value.
    """
    # 真实数据标签（1）和假数据标签（0）
    real_labels = tf.ones((real_data.shape[0], 1))
    fake_labels = tf.zeros((fake_data.shape[0], 1))

    # 合并真实数据和假数据及其标签
    inputs = tf.concat([real_data, fake_data], axis=0)
    labels = tf.concat([real_labels, fake_labels], axis=0)

    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = BinaryCrossentropy(from_logits=False)(labels, predictions)

    # 计算梯度并更新模型参数
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return loss.numpy()  # 返回损失值



In [30]:
def train_client_model(client_model, discrim_model, train_batch, optimizer):
    """
    Train the client model so that the discriminator cannot distinguish
    between real (val_batch) and fake (generated by client_model).
    
    Args:
    client_model (tf.keras.Model): The client model to be trained.
    discrim_model (tf.keras.Model): The discriminator model.
    train_batch (tf.Tensor): Batch of real data.
    optimizer (tf.keras.optimizers.Optimizer): Optimizer for the client model.
    
    Returns:
    float: The loss value.
    """
    with tf.GradientTape() as tape:
        # 通过 client_model 生成的输出
        generated_data = client_model(train_batch, training=True)
        
        # 通过 discrim_model 计算生成数据的判别值，确保 discrim_model 处于评估模式
        fake_outputs = discrim_model(generated_data, training=False)
        
        # 计算损失，使得 discrim_model 无法区分 generated_data 和真实数据
        loss = -tf.reduce_mean(tf.math.log(1 - fake_outputs + 1e-8))  # 1e-8 防止 log(0)
    
    # 计算 client_model 的梯度并更新
    gradients = tape.gradient(loss, client_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, client_model.trainable_variables))
    
    return loss.numpy()

In [31]:
from torch.utils.data import DataLoader
from itertools import cycle

In [32]:
def augment(image, label):
    # 随机水平翻转图像
    image = tf.image.random_flip_left_right(image)
    # 随机调整亮度
    image = tf.image.random_brightness(image, max_delta=0.1)
    # 确保图像值仍然在0到1的范围内
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, label

In [33]:
num_epochs = 20 # 定义训练的轮数
batch_size = 64  # 或者根据你的内存限制调整



train_dataset = tf.data.Dataset.from_tensor_slices(x_train).batch(64)
val_dataset = tf.data.Dataset.from_tensor_slices(x_val_tigger).batch(32).repeat()




discrim_model.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])
client_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
optimizer = Adam(learning_rate=0.0002, beta_1=0.5)


combined_dataset = tf.data.Dataset.zip((train_dataset, val_dataset))

# 使用 itertools.cycle 让验证数据循环使用
#val_loader = cycle(val_loader)
for epoch in range(num_epochs):
    for (train_batch, val_batch) in combined_dataset:
        real_data = client_model(train_batch)
        fake_data = client_model_1(val_batch)
        loss = train_discriminator(discrim_model, real_data, fake_data, optimizer)
        loss = train_client_model(client_model, discrim_model, train_batch, client_optimizer)
    print(f"Epoch: {epoch}, Loss: {loss}")
    

Epoch: 0, Loss: 0.06552637368440628
Epoch: 1, Loss: 1.7552763893036172e-05
Epoch: 2, Loss: 0.0001493968302384019
Epoch: 3, Loss: 0.010892369784414768
Epoch: 4, Loss: 0.0005643951008096337
Epoch: 5, Loss: 0.025559619069099426
Epoch: 6, Loss: 0.13535279035568237
Epoch: 7, Loss: 0.015224569477140903
Epoch: 8, Loss: 0.010880153626203537
Epoch: 9, Loss: 8.097632417047862e-06
Epoch: 10, Loss: 0.005207925569266081
Epoch: 11, Loss: 0.016737664118409157
Epoch: 12, Loss: 0.040154606103897095
Epoch: 13, Loss: 0.003724738024175167
Epoch: 14, Loss: 0.0025894923601299524
Epoch: 15, Loss: 0.2975025177001953
Epoch: 16, Loss: 0.42290574312210083
Epoch: 17, Loss: 0.02158060297369957
Epoch: 18, Loss: 0.004886220674961805
Epoch: 19, Loss: 0.3580864369869232


In [34]:
combined_model_mian_c = create_combined_model(client_model, server_model, compile_only=True)

In [35]:
loss, accuracy = combined_model_mian_c.evaluate(x_test, y_test, verbose=1)

313/313 [==============================] - 2s 7ms/step - loss: 3.8975 - accuracy: 0.3874


In [36]:
loss, accuracy = combined_model_mian_c.evaluate(x_test_tigger, y_test_tigger, verbose=1)

313/313 [==============================] - 2s 6ms/step - loss: 1.2104 - accuracy: 0.8380


In [ ]:
#313/313 [==============================] - 1s 3ms/step - loss: 0.9457 - accuracy: 0.7466
#Test loss: 0.9456651210784912, Test accuracy: 0.7465999722480774


## 